# 5.1 Introduction to convnets (畳み込みネットワーク)

- 基本的に，["Deep Learning with Python"](https://www.amazon.com/Deep-Learning-Python-Francois-Chollet/dp/1617294438/)のSection 5.1(120-129ページ)を元にしている．  
- githubで入手可能な[原本のコード](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.1-introduction-to-convnets.ipynb)を解説を兼ねて多少改編している．  

## (1) Convolution layers

２章では，MNISTデータセットを識別するために`Dense`層を２つを使ったが，ここでは単純な畳み込みを利用したネットワークを利用してみる．たとえ極めて単純なネットワークと言え，その精度は`Dense`より良い．  

次のネットワークは`Conv2D`と`MaxPooling2D`で構成されている．  
- 畳み込み層は入力として`(image_height, image_width, image_channels)`の形のテンソルを受け入れる  
  (バッチ数は含まれていない)．  
- ここでは，MNISTイメージである`(28, 28, 1)`を入力として受け入れる．  
  そこで，引数として`input_shape=(28, 28, 1)`を入れる．  
- `Conv2D`や`MaxPooling2D`の出力は`(height, width, channels)`の形をした3Dテンソルであり，層が深くなるにつれて縦横が小さくなり，チャンネルは深くなっていくのが分かる．出力のチャンネル数は`Conv2D`の最初の引数で指定される(32か64)．  

In [1]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


In [2]:
## display the architecture of convnet
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


## (2) Dense layers

- 次に，分類を行うために出力の`(3, 3, 64)`テンソルを馴染みのある`Dense`層に伝える．  
- 出力が3Dテンソルであるため，1Dのテンソルに変換しなければならない．そこで，まず3Dテンソルを1Dに平坦化し，その後，`Dense`層を追加する．  
  + 10個(手書き数字)の分類を行うため，最終出力層での活性化関数として`softmax`を用いる．  
  + 畳み込み層からの`(3,3,64)`の出力は`Dense`層に移る前に`(576,)`に平坦化される．  

In [3]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax')) ## 10-way classification

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

## データの用意と訓練

２章で使ったコードを使いまわし，MNISTデータセットを訓練させる．

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 5s 81us/step - loss: 0.1774 - accuracy: 0.9440
Epoch 2/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.0474 - accuracy: 0.9854
Epoch 3/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.0320 - accuracy: 0.9900
Epoch 4/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.0237 - accuracy: 0.9927
Epoch 5/5
60000/60000 [==============================] - 4s 61us/step - loss: 0.0188 - accuracy: 0.9941


## 評価

`Dense`を使ったネットワークに比べ，畳み込みネットワークにでより良い精度(~=99%)が達成できる．

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

10000/10000 [==============================] - 1s 57us/step
0.9919000267982483


(end)